# Сравнение ручного и автоматического заполнения пропусков с помощью моделей

![](https://loginom.ru/sites/default/files/social_images/missing-data_1200x630.jpg)

В этой статье мы разберем, оценим и сравним методы заполнения пропусков в данных.

## Допущения

Будем использовать датасет

 - https://www.kaggle.com/datasets/mirichoi0218/insurance

Описание данных:

 - age: age of primary beneficiary
 - sex: insurance contractor gender, female, male
 - bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
 - objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
 - children: Number of children covered by health insurance / Number of dependents
 - smoker: Smoking
 - region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
 - charges: Individual medical costs billed by health insurance

Сразу отмечу, что восстанавливать целевой признак таким способом - не лучшая идея. Записи с пропуском целевого признака лучше удалить из выборки. Но если все таки рискнуть восстановить, то есть шанс улучшить модель или провести более точный анализ данных :)
Предположим, что удалять данные из выборки никак нельзя в силу каких-либо причин. Единственный выход - восстанавливать.

## Методика оценки и сравнения

На старте исследования мы имеем датасет без пропусков. Датасет имеет как целые, так и вещественные признаки.
Удаляем 20% рандомных данных - это и будет наш тест.
Оценивать будем с помощью conflusion_matrix и f1 для целых чисел и r2 для вещественных.

Ручное заполнение пропусков будет производится с помощью следующих методов:
 - заполнение данных средним
 - заполнение данных медианой
 - заполнение данных самым частотным значением класса
 - заполнение данных новым классом _unknown_
 - заполнение данных предыдущим значением (годится например, для временных рядов)

Для заполнения данных с помощью моделей будет использовать следующие модели:
 - линейную регрессию (Ridge)
 - модель градиентного бустинга (Catboost)
 - случайный лес (sklearn RandomForest)
 - модель ближайших соседей (Knei)

Смоделируем ситуацию, что пропуски у нас не случайны, удалим данные определенным образом. Посмотрим как с этим справится модель.

In [1]:
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from catboost import CatBoostRegressor, CatBoostClassifier

from sklearn.metrics import f1_score, confusion_matrix, r2_score

import plotly.express as px
import plotly.graph_objects as go

In [32]:
'y'.isalpha()

True

In [93]:
data = pd.read_csv('insurance.csv')
data_dummies = pd.get_dummies(data[['sex', 'smoker', 'region', ]], drop_first=True)
data = pd.concat([data, data_dummies], axis=1).drop(['sex', 'smoker', 'region', ], axis=1)
data.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0


In [94]:
data.shape

(1338, 9)

## Для начала решим задачу восстановления пропусков целых чисел

In [95]:
drop_indexes = data.sample(frac=.2).index
test = data.iloc[drop_indexes, 0]

In [96]:
data.iloc[drop_indexes, 0] = np.NaN
data['age'] = data['age'].fillna(data.groupby(by=['sex_male', 'children'])['age'].transform('mean').astype('int'))
pred = data.iloc[drop_indexes, 0]
f1_score(test, pred, average='micro')

0.02238805970149254

In [97]:
data.iloc[drop_indexes, 0] = np.NaN
data['age'] = data['age'].fillna(data.groupby(by=['sex_male', 'children'])['age'].transform('median').astype('int'))
pred = data.iloc[drop_indexes, 0]
f1_score(test, pred, average='micro')

0.033582089552238806

In [98]:
data.iloc[drop_indexes, 0] = np.NaN
data['age'] = data['age'].fillna(data['age'].mode()[0])
pred = data.iloc[drop_indexes, 0]
f1_score(test, pred, average='micro')

0.03731343283582089

In [99]:
data.iloc[drop_indexes, 0] = np.NaN
data['age'] = data['age'].bfill()
pred = data.iloc[drop_indexes, 0]
f1_score(test, pred, average='micro')

0.018656716417910446

In [142]:
data.iloc[drop_indexes, 0] = np.NaN

imputer = IterativeImputer()
data['age'] = imputer.fit_transform(data.drop(['bmi', 'charges'], axis=1)).astype('int')
pred = data.iloc[drop_indexes, 0]
pred = pred.rename('age_pred')
f1_score(test, pred, average='micro')

0.029850746268656716

In [143]:
from sklearn.metrics import mean_squared_error, max_error

np.sqrt(mean_squared_error(test, pred)), max_error(test, pred)

(14.489960497664601, 26)

In [144]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=test.index, y=test,  mode='markers', name='Test'))

fig.add_trace(go.Scatter(x=pred.index, y=pred, mode='markers', name='Pred'))

fig.show()